# GRPO Training for Italian Exercise Generator

## Setup

In [ ]:
# --- Cell 1: Setup and Imports ---

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Navigate to your project directory
# Make sure this path is correct for your Google Drive setup
import os
project_path = '/content/drive/MyDrive/Colab Notebooks/italian_teacher'
os.chdir(project_path)
print(f"Changed directory to: {os.getcwd()}")

# trl imported from local fork
# !pip install -e /content/drive/MyDrive/Colab\ Notebooks/trl

# Install dependencies (now includes google-generativeai for Gemini API)
!pip install -q trl transformers accelerate peft datasets spacy sentence-transformers bitsandbytes json5 openai google-generativeai tqdm nest_asyncio
!python -m spacy download it_core_news_sm
!pip install anthropic groq

# Standard library imports
import json
import random
from getpass import getpass

# Third-party imports
import torch
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import GRPOConfig, GRPOTrainer

# Local module imports
from src.rl.multi_reward_async import create_async_multi_reward
from src.rl.prompt_formatter import format_prompt_with_chat_template
from src.rl.reward_function import ExerciseRewardFunction

# Environment setup
os.environ["WANDB_DISABLED"] = "true"

print("\n--- Environment Setup ---")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Changed directory to: /content/drive/MyDrive/Colab Notebooks/italian_teacher
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 31.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('it_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.

--- Environment Setup ---
PyTorch version: 2.8.0+cu126
CUDA available: True
GPU: NVIDIA A100-SXM4-80GB


In [ ]:
# --- Cell 2: Configuration ---
# All training parameters are here for easy modification.

BASE_MODEL_PATH = "./models/italian_v8_grpo_round2"  # Input model for this training run
OUTPUT_DIR = "./models/TeacherPet_italian_grpo"      # Where the new model will be saved
NUM_SAMPLES = 800                                    # Number of training requests to use
RANDOM_SEED = 44                                      # Seed for reproducibility

# Scorer settings
DISABLED_SCORERS = []          # No scorers disabled
FLUENCY_USE_LLM = False        # Use rule-based checks only (fast, free)

# --- GRPO Configuration ---
grpo_config = GRPOConfig(
    output_dir=OUTPUT_DIR,
    num_train_epochs=1,
    per_device_train_batch_size=12,  # Reduce from 12
    gradient_accumulation_steps=6,  # Increase to keep same effective batch size
    learning_rate=5e-6,  # Increase from 5e-6
    warmup_steps=50,
    logging_steps=5,
    save_steps=25,
    save_total_limit=3,
    bf16=True,
    remove_unused_columns=False,
    report_to="none",

    # GRPO-specific generation settings
    num_generations=4,  # Keep this
    max_prompt_length=1024,
    max_completion_length=350,
    temperature=0.9,  # Increase for diversity
    generation_batch_size=12,
    beta=0.05,

    # Generation kwargs - CRITICAL FIX
    generation_kwargs={
        "bos_token_id": 128000,
        "do_sample": True,  # ✅ MUST BE TRUE!
        "eos_token_id": [128009, 128001],
        "temperature": 0.8,
        "top_p": 0.9,
        "top_k": 50,
        "padding_side": "left",

    }
)

print("✅ Configuration loaded.")
print(f"   Base model: {BASE_MODEL_PATH}")
print(f"   Output directory: {OUTPUT_DIR}")
print(f"   Training samples: {NUM_SAMPLES}")
print(f"   Max prompt length: 2048 tokens (prevents truncation)")
print(f"   Fluency scorer: {'Enabled (rule-based)' if not DISABLED_SCORERS else 'Disabled'}")

✅ Configuration loaded.
   Base model: ./models/italian_v8_grpo_round2
   Output directory: ./models/TeacherPet_italian_grpo
   Training samples: 1200
   Max prompt length: 2048 tokens (prevents truncation)
   Fluency scorer: Enabled (rule-based)


In [ ]:
# --- Cell 3: Helper Functions & Main Execution ---

def load_secrets_from_file():
    """
    Load API keys from .secrets.json file if it exists.
    Checks multiple locations: Google Drive root, then current directory.
    """
    from pathlib import Path

    secrets_paths = [
        Path.home() / "Google Drive" / "My Drive" / ".secrets.json",  # Local path
        Path("/content/drive/My Drive/.secrets.json"),                # Colab path
        Path('.secrets.json')                                         # Current directory
    ]

    for path in secrets_paths:
        if path.exists():
            print(f"✅ Loading API keys from {path}")
            with open(path, 'r') as f:
                secrets = json.load(f)

            loaded_keys = []
            for key, value in secrets.items():
                if value and value not in ["your-openai-key-here", "your-google-key-here", ""]:
                    os.environ[key] = value
                    loaded_keys.append(key)

            if loaded_keys:
                print(f"   Loaded {len(loaded_keys)} API key(s)")
                return True

    print("⚠️  No .secrets.json found. Make sure API keys are in Colab secrets or environment.")
    return False


def load_training_data(tokenizer, num_samples: int, seed: int):
    """Load or generate training requests and prepare dataset."""
    requests_path = "src/rl/training_requests.json"

    if os.path.exists(requests_path):
        print(f"Loading existing training requests from {requests_path}...")
        with open(requests_path, "r") as f:
            training_requests = json.load(f)
    else:
        from src.rl.generate_training_requests import generate_training_requests
        print(f"Generating {num_samples} new training requests...")
        training_requests = generate_training_requests(
            num_requests=num_samples,
            output_path=requests_path
        )

    print(f"✅ Loaded {len(training_requests)} training requests.")

    # Format prompts
    prompts = [
        format_prompt_with_chat_template(req, tokenizer, add_examples=True)
        for req in training_requests
    ]

    # Sample if needed
    if len(prompts) > num_samples:
        print(f"Sampling {num_samples} requests (seed={seed})...")
        random.seed(seed)
        random_indices = random.sample(range(len(prompts)), num_samples)
        prompts = [prompts[i] for i in random_indices]
        training_requests = [training_requests[i] for i in random_indices]

    return Dataset.from_dict({
        "prompt": prompts,
        "request": training_requests,
    })


print("=" * 80)
print("🚀 STARTING GRPO TRAINING")
print("=" * 80)

# Load API keys
print("\n--- Loading Secrets ---")
load_secrets_from_file()

# Load Model and Tokenizer
print(f"\n--- Loading Model ---")
print(f"Base model: {BASE_MODEL_PATH}")
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_PATH, padding_side='left')
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_PATH,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
    use_cache=False,
)
model.gradient_checkpointing_enable()
model.config.pad_token_id = tokenizer.pad_token_id
model.config.padding_side = tokenizer.padding_side
print("✅ Model and tokenizer loaded.")

# Prepare Training Data
print("\n--- Preparing Training Data ---")
train_dataset = load_training_data(tokenizer, num_samples=NUM_SAMPLES, seed=RANDOM_SEED)

# Initialize Reward Function
print("\n--- Initializing Reward Function ---")
reward_fn_instance = ExerciseRewardFunction(
    device="cuda",
    disabled_scorers=DISABLED_SCORERS,
    fluency_use_llm=FLUENCY_USE_LLM,
    concurrency_limit=5  # High concurrency for speed
)
reward_func = create_async_multi_reward(reward_fn_instance, use_openai=True)
print("✅ Reward function ready.")

print(f'Tokenizer type: {type(tokenizer)}')
print(f'Padding side: {tokenizer.padding_side}')
if hasattr(tokenizer, 'tokenizer'):
    print(f'Has sub-tokenizer: {tokenizer.tokenizer.padding_side}')


🚀 STARTING GRPO TRAINING

--- Loading Secrets ---
✅ Loading API keys from /content/drive/My Drive/.secrets.json
   Loaded 9 API key(s)

--- Loading Model ---
Base model: ./models/italian_v8_grpo_round2


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

✅ Model and tokenizer loaded.

--- Preparing Training Data ---
Loading existing training requests from src/rl/training_requests.json...
✅ Loaded 2000 training requests.
Sampling 1200 requests (seed=44)...

--- Initializing Reward Function ---
Loading spaCy model: it_core_news_sm...
✅ spaCy model loaded
Reward function will use device: cuda
     ✅ Gemini: 4 API key(s)
     ✅ OpenAI: configured
     ✅ Anthropic: configured
     ✅ Groq: configured
     ✅ DeepSeek: configured
     ✅ Cerebras: configured
  ✅ LLM API Handler initialized
     Providers: gemini, openai, anthropic, groq, deepseek, cerebras
     Total models: 12
Initializing scorers...
  ✅ LLM scoring enabled for cefr_alignment (batch size: 10)
  ✅ LLM scoring enabled for fluency (batch size: 10)
  ✅ LLM scoring enabled for grammar_correctness (batch size: 10)
  ✅ LLM scoring enabled for coherence (batch size: 10)
Loading sentence transformer for topic similarity...
✅ Sentence transformer loaded in cuda
  ✅ LLM topic checking en

In [ ]:
# --- Add Validation Tracking ---
from src.rl.validation_callback import ValidationCallback, select_validation_samples
from src.rl.prompt_formatter import format_prompt_with_chat_template

# Select 10 diverse validation samples
validation_samples = select_validation_samples(
    training_requests_path="src/rl/training_requests.json",
    num_samples=10,  # Adjust this number (5-15 recommended)
    seed=RANDOM_SEED
)

# Format prompts for validation samples
validation_prompts = [
    format_prompt_with_chat_template(req, tokenizer, add_examples=True)
    for req in validation_samples
]

# Create validation callback
validation_callback = ValidationCallback(
    validation_samples=validation_samples,
    validation_prompts=validation_prompts,
    reward_function=reward_func,
    tokenizer=tokenizer,
    output_dir=OUTPUT_DIR,
    num_generations=3  # Generate 3 completions per sample for comparison
)

print("✅ Validation callback ready.")


📊 Selecting 10 validation samples from src/rl/training_requests.json...
   Total requests available: 2000
   Exercise types: unknown
   ✅ Selected 10 diverse samples
   Distribution: {'unknown': 10}

📊 Validation Callback initialized:
   10 validation samples
   3 generations per sample
   Results will be saved to: models/TeacherPet_italian_grpo/validation_results
✅ Validation callback ready.


In [ ]:
import trl
print(f"TRL version: {trl.__version__}")


TRL version: 0.24.0


In [6]:
# Initialize Trainer
print("\n--- Initializing GRPO Trainer ---")
trainer = GRPOTrainer(
    model=model,
    args=grpo_config,
    reward_funcs=reward_func,
    train_dataset=train_dataset,
    processing_class=tokenizer,
    callbacks=[validation_callback],
)

# After: trainer = GRPOTrainer(...)
print(f"\n🔍 GRPO Generation Settings:")
print(f"   num_generations (config): {grpo_config.num_generations}")
print(f"   generation_batch_size: {grpo_config.generation_batch_size}")
if hasattr(trainer, 'generation_config'):
    print(f"   trainer.generation_config: {trainer.generation_config}")
print("✅ GRPO Trainer initialized.")


# Start Training
print("\n" + "=" * 80)
print("🔥 TRAINING BEGINS")
print("=" * 80)
trainer.train()
print("\n" + "=" * 80)
print("🎉 TRAINING COMPLETE")
print("=" * 80)

# Save Final Model
print(f"\n--- Saving Model ---")
print(f"Output directory: {OUTPUT_DIR}")
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print("✅ Model saved successfully.")

The model is already on multiple devices. Skipping the move to device specified in `args`.



--- Initializing GRPO Trainer ---

🔍 GRPO Generation Settings:
   num_generations (config): 3
   generation_batch_size: 12
   trainer.generation_config: GenerationConfig {
  "bos_token_id": 128000,
  "do_sample": true,
  "eos_token_id": [
    128009,
    128001
  ],
  "max_new_tokens": 350,
  "pad_token_id": 128009,
  "padding_side": "left",
  "temperature": 0.8,
  "top_k": null,
  "top_p": 0.9
}

✅ GRPO Trainer initialized.

🔥 TRAINING BEGINS


`generation_config` default values have been modified to match model-specific defaults: {'max_length': 8192}. If this is not desired, please set these values explicitly.


   [Call #1 for step 0: scoring 12 completions]

⏳ Step 1/3: Parsing 12 JSON completions...


   Parse stats: 12/12 valid JSON (100.0%), 0 empty, 0 failed → 12 scorable
⏳ Step 2/3: Scoring 12 completions with batched reward function...


...


🎯 Reward calculation complete (0.0s):
   TOTAL     : min=0.000, max=0.000, avg=0.000
    Rewards: avg=0.00, max=0.00, min=0.00

  Generating for validation sample 10/10...


   [Call #10 for step 0: scoring 1 completions]

⏳ Step 1/3: Parsing 1 JSON completions...


   Parse stats: 0/1 valid JSON (0.0%), 0 empty, 1 failed → 0 scorable
⏳ Step 2/3: Scoring 1 completions with batched reward function...

   ❌ WARNING: 0 successful LLM calls!
      0 models in cooldown, 0 at daily limit
      💡 TIP: Likely hitting RPM (requests/minute) limits
      💡 Solution: Reduce batch size or add request pacing
⏳ Step 3/3: Computing CPU-bound rewards and aggregating results...



🎯 Reward calculation complete (0.0s):
   TOTAL     : min=0.000, max=0.000, avg=0.000
    Rewards: avg=0.00, max=0.00, min=0.00



📊 Validation Results at Step 50:
   Average Reward: 0.00
   Max Reward:     0.00
   Min Reward:     0.00
   Saved to: models/TeacherPet_italian_grpo/validation_results/validation_step_50.json
   ➡️ Change from step 25: +0.00


🎉 TRAINING COMPLETE

--- Saving Model ---
Output directory: ./models/TeacherPet_italian_grpo
✅ Model saved successfully.
